In [0]:
%sql
select 
  count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
   end) as day_of_week 
  from workspace.instacart.orders
 group by order_dow 
 order by order_dow asc

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from workspace.instacart.orders
 group by order_hour_of_day 
 order by order_hour_of_day

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT d.department, COUNT(DISTINCT p.product_id) AS products
FROM (
  SELECT product_id, try_cast(department_id AS BIGINT) AS department_id
  FROM instacart.products
) p
INNER JOIN instacart.departments d
  ON d.department_id = p.department_id
GROUP BY d.department
ORDER BY products DESC
LIMIT 10


Databricks visualization. Run in Databricks to view.

In [0]:
# Organize the data by shopping basket
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, o.order_id from workspace.instacart.products p inner join workspace.instacart.order_products_prior o where o.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')
display(baskets)